### 0. 모델 로드 및 전처리 등 실험과 상관 없는 내용

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models,datasets, transforms
from mevit_model import MultiExitViT
from tqdm import tqdm
from itertools import combinations
import numpy as np
from scipy.stats import entropy
from temperature_scaling import TemperatureScaling
from Dloaders import Dloaders
####################################################################
IMG_SIZE = 224
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_name = {'cifar10':datasets.CIFAR10, 'cifar100':datasets.CIFAR100,'imagenet':None}
dataset_outdim = {'cifar10':10, 'cifar100':100,'imagenet':1000}
##############################################################
################ 0. Hyperparameters ##########################
##############################################################
batch_size = 1024
data_choice='cifar100'
mevit_isload=True
mevit_pretrained_path=f"models/{data_choice}/integrated_ee.pth"

backbone_path=f'models/{data_choice}/vit_{data_choice}_backbone.pth'
start_lr=1e-4
max_iter=200

ee_list=[0,1,2,3,4,5,6,7,8,9]#exit list ex) [0,1,2,3,4,5,6,7,8,9]
exit_loss_weights=[1,1,1,1,1,1,1,1,1,1,1]#exit마다 가중치
exit_num=11
##############################################################
dloaders=Dloaders(data_choice=data_choice,batch_size=batch_size,IMG_SIZE=IMG_SIZE)
train_loader,test_loader = dloaders.get_loaders()

# Load the pretrained ViT model from the saved file
pretrained_vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

if data_choice != 'imagenet':
    pretrained_vit.heads.head = nn.Linear(pretrained_vit.heads.head.in_features, dataset_outdim[data_choice])  # Ensure output matches the number of classes

    # Load model weights
    pretrained_vit.load_state_dict(torch.load(backbone_path))
    pretrained_vit = pretrained_vit.to(device)
#from torchinfo import summary
#summary(pretrained_vit,input_size= (64, 3, IMG_SIZE, IMG_SIZE))

model = MultiExitViT(pretrained_vit,num_classes=dataset_outdim[data_choice],ee_list=ee_list,exit_loss_weights=exit_loss_weights).to(device)

# Assume a pretrained model (replace with your own model)
model.load_state_dict(torch.load(mevit_pretrained_path))  # Load your trained weights
file_path = f'cache_result_mevit_{data_choice}.pt'

Files already downloaded and verified
Files already downloaded and verified


### 1. MEVIT의 각 출구들에서의 Precision을 확인

In [ ]:
cifar10_accs = [72.76, 80.35, 85.8, 89.42, 91.98, 93.12, 94.94, 96.44, 96.95, 97.35, 97.63]
cifar100_accs = [48.21, 59.63, 67.93, 73.31, 77.9, 80.8, 83.71, 85.33, 86.61, 87.04, 87.75]
imagenet_accs = []

### 2. MEVIT의 각 출구들까지의 FLOPs 확인

In [ ]:
# from torchinfo import summary
# model.set_each_ee_test_mode(10)
# summary(model,input_size= (1, 3, IMG_SIZE, IMG_SIZE))
FLOPs_list=[125.29, 130.01, 134.74, 139.46, 144.19, 148.91, 153.64, 158.36, 163.09, 167.82, 172.54]